In [ ]:
from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import math
import random
import json
from collections import defaultdict
from scipy.spatial import distance
from ultralytics import YOLO
import uvicorn
import nest_asyncio

nest_asyncio.apply()

app = FastAPI()

video_url = "http://192.168.69.91:8080/video"
cap = cv2.VideoCapture(video_url)

model = YOLO("../models/first_model.pt")
LABEL_COLORS = {}
allowed_pairs = {("tank", "friend"), ("enemy", "friend"), ("enemy", "wall"), ("friend", "wall")}

class ObjectTracker:
    def __init__(self):
        self.objects = {}
        self.type_counters = defaultdict(int)

    def get_unique_name(self, class_name):
        self.type_counters[class_name] += 1
        return f"{class_name.capitalize()}{self.type_counters[class_name]}"

    def track_object(self, center_x, center_y, class_name, frame_counter):
        for obj_id, obj_info in self.objects.items():
            if (obj_info['type'] == class_name and 
                distance.euclidean((obj_info['last_x'], obj_info['last_y']), (center_x, center_y)) < 50):
                obj_info.update({'last_x': center_x, 'last_y': center_y, 'last_seen': frame_counter, 'status': 'Active'})
                return obj_id

        unique_name = self.get_unique_name(class_name)
        self.objects[unique_name] = {'type': class_name, 'last_x': center_x, 'last_y': center_y, 'last_seen': frame_counter, 'status': 'Active'}
        return unique_name

    def clean_old_objects(self, frame_counter):
        for obj_id, obj_info in list(self.objects.items()):
            if frame_counter - obj_info['last_seen'] > 10:
                obj_info['status'] = 'Dead'

object_tracker = ObjectTracker()

def get_label_color(label):
    if label not in LABEL_COLORS:
        LABEL_COLORS[label] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    return LABEL_COLORS[label]

def generate_frames():
    frame_counter = 0
    while True:
        success, frame = cap.read()
        if not success:
            break

        frame_counter += 1
        centers = []
        results = model(frame)
        detections = results[0].boxes  

        for box in detections:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label_id = int(box.cls[0])
            class_name = model.names[label_id].lower()
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            obj_id = object_tracker.track_object(center_x, center_y, class_name, frame_counter)

            color = get_label_color(label_id)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, obj_id, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            centers.append((center_x, center_y, class_name, obj_id))

        for i in range(len(centers)):
            for j in range(i + 1, len(centers)):
                p1_x, p1_y, label1, name1 = centers[i]
                p2_x, p2_y, label2, name2 = centers[j]
                if (label1, label2) in allowed_pairs or (label2, label1) in allowed_pairs:
                    distance_pixels = int(math.dist((p1_x, p1_y), (p2_x, p2_y)))
                    cv2.line(frame, (p1_x, p1_y), (p2_x, p2_y), (255, 255, 0), 2)
                    midpoint = ((p1_x + p2_x) // 2, (p1_y + p2_y) // 2)
                    cv2.putText(frame, f"{distance_pixels}px", midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

        object_tracker.clean_old_objects(frame_counter)

        _, buffer = cv2.imencode(".jpg", frame)
        frame_bytes = buffer.tobytes()

        yield (b"--frame\r\n"
               b"Content-Type: image/jpeg\r\n\r\n" + frame_bytes + b"\r\n")

@app.get("/video_feed")
def video_feed():
    return StreamingResponse(generate_frames(), media_type="multipart/x-mixed-replace; boundary=frame")

if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [47424]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:55788 - "GET /video_feed?t=1738662694553 HTTP/1.1" 200 OK

INFO:     127.0.0.1:55789 - "GET /video_feed?t=1738662694768 HTTP/1.1" 200 OK

0: 384x640 1 Dead soldier -enemy-, 1 Dead soldier -frd-, 1 Tank, 2 Walls, 1 enemy, 44.1ms
Speed: 1.8ms preprocess, 44.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Dead soldier -enemy-, 1 Dead soldier -frd-, 1 Tank, 2 Walls, 1 enemy, 57.8ms
Speed: 2.6ms preprocess, 57.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Dead soldier -enemy-, 1 Dead soldier -frd-, 1 Tank, 2 Walls, 1 enemy, 54.4ms
Speed: 2.2ms preprocess, 54.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Dead soldier -enemy-, 1 Dead soldier -frd-, 1 Tank, 3 Walls, 1 enemy, 53.3ms
Speed: 1.9ms preprocess, 53.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Dead soldier -enemy-, 1 Dead soldier -frd-, 1 Tank, 2 Walls, 1 enemy, 44.3ms


[mjpeg @ 0x153c89670] overread 1



0: 384x640 1 Dead soldier -enemy-, 2 Dead soldier -frd-s, 1 Tank, 3 Walls, 1 enemy, 1 friend, 55.2ms
Speed: 1.4ms preprocess, 55.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Dead soldier -enemy-, 2 Dead soldier -frd-s, 1 Tank, 3 Walls, 1 enemy, 1 friend, 48.1ms
Speed: 2.1ms preprocess, 48.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Dead soldier -enemy-, 1 Dead soldier -frd-, 1 Tank, 3 Walls, 1 enemy, 1 friend, 44.9ms
Speed: 2.4ms preprocess, 44.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Dead soldier -enemy-, 2 Dead soldier -frd-s, 1 Tank, 3 Walls, 1 enemy, 1 friend, 48.2ms
Speed: 1.6ms preprocess, 48.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Dead soldier -enemy-, 2 Dead soldier -frd-s, 1 Tank, 3 Walls, 1 enemy, 1 friend, 43.5ms
Speed: 1.5ms preprocess, 43.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 38

[mpjpeg @ 0x153c79d70] Expected boundary '--' not found, instead found a line of 100 bytes
[mpjpeg @ 0x153c79d70] Expected boundary '--' not found, instead found a line of 6 bytes


...

[mpjpeg @ 0x153c79d70] Expected boundary '--' not found, instead found a line of 46 bytes


INFO:     127.0.0.1:55853 - "GET /video_feed?t=1738662808792 HTTP/1.1" 200 OK


[mpjpeg @ 0x153c79d70] Expected boundary '--' not found, instead found a line of 29 bytes


INFO:     127.0.0.1:55859 - "GET /video_feed?t=1738662842794 HTTP/1.1" 200 OK


[mpjpeg @ 0x153c79d70] Expected boundary '--' not found, instead found a line of 51 bytes


In [ ]:
from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import random
import math
import numpy as np
from ultralytics import YOLO
import uvicorn
import nest_asyncio
from collections import defaultdict

nest_asyncio.apply()

app = FastAPI()

video_url = "http://192.168.69.91:8080/video"
cap = cv2.VideoCapture(video_url)

model = YOLO("../models/first_model.pt")
LABEL_COLORS = {}
object_counters = defaultdict(int)

allowed_pairs = {
    ("tank", "friend"),
    ("enemy", "friend"),
    ("enemy", "wall"),
    ("friend", "wall")
}

def get_label_color(label):
    if label not in LABEL_COLORS:
        LABEL_COLORS[label] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    return LABEL_COLORS[label]

def generate_frames():
    while True:
        success, frame = cap.read()
        if not success:
            break

        results = model(frame)
        detections = results[0].boxes
        object_counters.clear()
        centers = []

        for box in detections:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0]
            label_id = int(box.cls[0])
            class_name = model.names[label_id]

            object_counters[class_name] += 1
            unique_label = f"{class_name} {object_counters[class_name]}"

            color = get_label_color(class_name)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            text = f"{unique_label} {confidence:.2f}"
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            centers.append((center_x, center_y, class_name.lower(), unique_label))
            cv2.circle(frame, (center_x, center_y), 5, color, -1)

            print(f"Detected: {unique_label}")

        for i in range(len(centers)):
            for j in range(i + 1, len(centers)):
                p1_x, p1_y, label1, name1 = centers[i]
                p2_x, p2_y, label2, name2 = centers[j]

                if (label1, label2) in allowed_pairs or (label2, label1) in allowed_pairs:
                    distance_pixels = int(math.dist((p1_x, p1_y), (p2_x, p2_y)))
                    cv2.line(frame, (p1_x, p1_y), (p2_x, p2_y), (255, 255, 0), 2)
                    midpoint = ((p1_x + p2_x) // 2, (p1_y + p2_y) // 2)
                    cv2.putText(frame, f"{distance_pixels}px", midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

        _, buffer = cv2.imencode(".jpg", frame)
        frame_bytes = buffer.tobytes()

        yield (b"--frame\r\n"
               b"Content-Type: image/jpeg\r\n\r\n" + frame_bytes + b"\r\n")

@app.get("/video_feed")
def video_feed():
    return StreamingResponse(generate_frames(), media_type="multipart/x-mixed-replace; boundary=frame")

if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [38945]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:61846 - "GET /video_feed?t=1738656133733 HTTP/1.1" 200 OK

0: 384x640 (no detections), 74.1ms
Speed: 3.9ms preprocess, 74.1ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 53.5ms
Speed: 3.0ms preprocess, 53.5ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 42.8ms
Speed: 2.0ms preprocess, 42.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 47.3ms
Speed: 7.0ms preprocess, 47.3ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 47.7ms
Speed: 1.4ms preprocess, 47.7ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 42.2ms
Speed: 2.0ms preprocess, 42.2ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 37.3ms
Speed: 1.4ms preprocess, 37.3ms inference, 0.3ms postprocess per image at shape (1, 3, 3

[mjpeg @ 0x14e918570] error dc
[mjpeg @ 0x14e918570] error y=56 x=58
[mjpeg @ 0x14e918570] overread 8
[mjpeg @ 0x14e918570] No JPEG data found in image


0: 384x640 1 Dead soldier -enemy-, 1 Tank, 2 Walls, 1 enemy, 2 friends, 60.6ms
Speed: 2.1ms preprocess, 60.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Detected: Tank 1
Detected: Dead soldier -enemy- 1
Detected: Wall 1
Detected: friend 1
Detected: enemy 1
Detected: Wall 2
Detected: friend 2

0: 384x640 1 Dead soldier -enemy-, 1 Tank, 2 Walls, 1 enemy, 2 friends, 64.0ms
Speed: 2.6ms preprocess, 64.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Detected: Tank 1
Detected: Dead soldier -enemy- 1
Detected: Wall 1
Detected: enemy 1
Detected: friend 1
Detected: Wall 2
Detected: friend 2
0: 384x640 1 Dead soldier -enemy-, 1 Tank, 2 Walls, 1 enemy, 2 friends, 68.7ms
Speed: 3.2ms preprocess, 68.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Detected: Tank 1
Detected: Dead soldier -enemy- 1
Detected: Wall 1
Detected: enemy 1
Detected: friend 1
Detected: Wall 2
Detected: friend 2

0: 384x640 1 Dead soldier -enemy-, 1 Tank, 2 Walls

[mjpeg @ 0x14e918570] overread 8


...

0: 384x640 (no detections), 50.5ms
Speed: 2.5ms preprocess, 50.5ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 48.9ms
Speed: 1.6ms preprocess, 48.9ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 48.0ms
Speed: 1.9ms preprocess, 48.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 44.5ms
Speed: 2.0ms preprocess, 44.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 39.9ms
Speed: 1.3ms preprocess, 39.9ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 39.8ms
Speed: 1.3ms preprocess, 39.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 35.0ms
Speed: 1.9ms preprocess, 35.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 36.6ms
Speed: 2.0ms preprocess, 36.6ms in

[mjpeg @ 0x14e918570] error count: 74
[mjpeg @ 0x14e918570] error y=17 x=86
[mjpeg @ 0x14e918570] No JPEG data found in image


Detected: Tank 1
Detected: Dead soldier -enemy- 1
Detected: Wall 1
Detected: friend 1
Detected: friend 2
Detected: enemy 1
Detected: Wall 2
INFO:     127.0.0.1:63334 - "GET /video_feed HTTP/1.1" 200 OK

0: 384x640 1 Dead soldier -enemy-, 1 Tank, 2 Walls, 1 enemy, 2 friends, 60.9ms
Speed: 3.7ms preprocess, 60.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Detected: Dead soldier -enemy- 1
Detected: Tank 1
Detected: Wall 1
Detected: Wall 2
Detected: friend 1
Detected: enemy 1
Detected: friend 2

0: 384x640 1 Dead soldier -enemy-, 1 Tank, 2 Walls, 1 enemy, 2 friends, 42.9ms
Speed: 2.9ms preprocess, 42.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Detected: Dead soldier -enemy- 1
Detected: Tank 1
Detected: Wall 1
Detected: Wall 2
Detected: friend 1
Detected: enemy 1
Detected: friend 2

0: 384x640 1 Dead soldier -enemy-, 1 Tank, 2 Walls, 1 enemy, 2 friends, 38.9ms
Speed: 1.3ms preprocess, 38.9ms inference, 0.9ms postprocess per image at shape (1, 

[mjpeg @ 0x14e918570] overread 8
[mjpeg @ 0x14e918570] No JPEG data found in image


0: 384x640 1 Dead soldier -enemy-, 1 Tank, 2 Walls, 1 enemy, 2 friends, 46.3ms
Speed: 1.8ms preprocess, 46.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Detected: Wall 1
Detected: friend 1
Detected: Dead soldier -enemy- 1
Detected: enemy 1
Detected: friend 2
Detected: Tank 1
Detected: Wall 2

0: 384x640 1 Dead soldier -enemy-, 1 Tank, 2 Walls, 1 enemy, 2 friends, 53.1ms
Speed: 1.8ms preprocess, 53.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Detected: friend 1
Detected: Wall 1
Detected: Dead soldier -enemy- 1
Detected: enemy 1
Detected: Tank 1
Detected: friend 2
Detected: Wall 2

0: 384x640 1 Dead soldier -enemy-, 1 Tank, 2 Walls, 1 enemy, 2 friends, 39.7ms
Speed: 1.3ms preprocess, 39.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Detected: Dead soldier -enemy- 1
Detected: Wall 1
Detected: friend 1
Detected: enemy 1
Detected: friend 2
Detected: Tank 1
Detected: Wall 2

0: 384x640 1 Dead soldier -enemy-, 1 Tank, 2 Wall

Note: you may need to restart the kernel to use updated packages.
